In [10]:
from glob import glob
import re

import ipywidgets as widgets
from IPython.display import display, HTML, Javascript, clear_output

from langchain.document_loaders import DirectoryLoader
from langchain.schema import Document
from langchain.vectorstores import Qdrant
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import LlamaForCausalLM, LlamaTokenizer, GenerationConfig
from peft import PeftModel


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 110
CUDA SETUP: Loading binary /opt/conda/envs/py310/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda110.so...


/opt/conda/envs/py310/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /opt/conda did not contain libcudart.so as expected! Searching further paths...
  warn(msg)


In [11]:
# print("Loading transformer embeddings...")
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

print("Loading Vicuna tokenizer")
tokenizer = LlamaTokenizer.from_pretrained("eachadea/vicuna-13b-1.1")

print("Loading Vicuna model")
model = LlamaForCausalLM.from_pretrained("eachadea/vicuna-13b-1.1", load_in_8bit=True, device_map="auto")
model = PeftModel.from_pretrained(model, "kmnis/DocScribe")

Loading transformer embeddings...


Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading Vicuna tokenizer
Loading Vicuna model


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [12]:
def get_results(question):
    prompt = f"""### Answer below question truthfully.

### Question:
{question}

### Answer:
"""
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()

    generation_config = GenerationConfig(temperature=0.6, top_p=0.95, repetition_penalty=1.15)

    generation_output = model.generate(input_ids=input_ids, generation_config=generation_config,
                                       return_dict_in_generate=True, output_scores=False, max_new_tokens=100)

    for out in generation_output.sequences:
        out = tokenizer.decode(out)
        out = out.split("### Answer:")[1].split("</s>")[0].strip()
        print(out + "\n")

In [4]:
question = "What are the symptoms of Hay fever?"
get_results(question)

The symptoms of hay fever include sneezing, itchy eyes, runny nose, and congestion.



In [5]:
question = "What factors increase the risk of developing pneumoconiosis?"
get_results(question)

The following factors can increase the risk of developing pneumoconiosis:

1. Exposure to dusts, such as those found in mining, construction, and agricultural work.
2. Prolonged exposure to harmful gases or fumes, such as those found in welding or metalworking.
3. Smoking.
4. Age (risk increases with age).
5. Previous history of lung disease or other occup



In [6]:
question = "What is the role of the ciliary muscle?"
get_results(question)

The ciliary muscle plays a crucial role in controlling the shape and movement of the lens within the eye. It is responsible for changing the curvature of the lens, which in turn changes the focus of the eye. When the ciliary muscle contracts, it causes the lens to become thicker and more curved, allowing the eye to focus on objects that are close by. When the muscle relaxes, the lens becomes flatter and allows



In [7]:
question = "What are the epidemiological and demographic characteristics of non-alcoholic fatty liver disease?"
get_results(question)

Non-alcoholic fatty liver disease (NAFLD) is a common condition worldwide, affecting approximately 20% to 30% of adults in developed countries [1]. The prevalence of NAFLD increases with obesity, insulin resistance, and type 2 diabetes mellitus [2]. In addition, NAFLD is more common in women than men, particularly in postmenopausal women [

